<a href="https://colab.research.google.com/github/fahmiajik12/Python/blob/Master/XrayPnumonia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pendahuluan

In [ ]:
#instalasi kaggel
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#mengambil dataset
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

In [ ]:
#mengekstrak dataset pada /content/
!unzip chest-xray-pneumonia.zip
!rm chest-xray-pneumonia.zip

melakukan import yang diperlukan, selain itu Salah satu praktik terbaik yang harus dilakukan saat melakukan project machine learning adalah dengan menentukan konstanta, sehingga memfasilitasi perubahan lebih lanjut. Mengingat itu, perlu dilakukan penentuan nilai bacth size, tinggi dan lebar gambar, dan *learning rate*.

In [ ]:
import os
import cv2
import seaborn as sns
import numpy as np 
import matplotlib.pyplot as plt
import tensorflow as tf
import datetime
from PIL import Image
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras.layers import add
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix

np.random.seed(777)
tf.random.set_seed(777)

In [ ]:
BATCH_SIZE = 32
IMG_HEIGHT = 240
IMG_WIDTH = 240
ALPHA = 1e-4

# Load Data

Kernel ini menggunakan dataset Chest X-Ray Images (Pneumonia), yang disusun menjadi 3 folder (train, test, val) dan berisi subfolder untuk setiap kategori gambar (Pneumonia / Normal). Ada 5.863 gambar X-Ray (JPEG) dan 2 kategori (Pneumonia / Normal).

Untuk analisis gambar Chest X-Ray, semua radiografi dada pada awalnya diskrining untuk kontrol kualitas dengan menghapus semua pindaian berkualitas rendah atau tidak terbaca. 

In [ ]:
data_dir = '/content/chest_xray'

In [ ]:
labels = ['NORMAL','PNEUMONIA']
def get_data(data_dir):
    data = [] 
    for label in labels: 
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE) 
                resized_arr = cv2.resize(img_arr, (IMG_WIDTH, IMG_HEIGHT))
                data.append([resized_arr, class_num])
            except Exception as e:
                pass

    return np.array(data)

kemudian kita definisikan sebuah fungsi untuk mengembalikan np.array dengan semua gambar yang terletak di direktori tertentu dan menggunakannya untuk memuat data training, validation dan test data

In [ ]:
train = get_data('/content/chest_xray/train/')
test = get_data('/content/chest_xray/test/')
val = get_data('/content/chest_xray/val/')

kemudian perlu juga untuk melihat berapa banyak gambar yang dimiliki dari setiap class di training set. Selain itu, mari kita lihat bagaimana gambar didistribusikan diantara training, validation dan test data

In [ ]:
print(f"{[y for _, y in train].count(0)} NORMAL IMAGES IN TRAINING SET")
print(f"{[y for _, y in train].count(1)} PNEUMONIA IMAGES IN TRAINING SET")

In [ ]:
print(f'Images in TRAINING SET: {train.shape[0]}')
print(f'Images in VALIDATION SET: {val.shape[0]}')
print(f'Images in TEST SET: {test.shape[0]}')

Seperti yang dapat dilihat pada output dari dua cell sebelumnya, terdapat masalah data yang tidak seimbang dan dengan proporsi yang agak aneh antara training set dan validation set.

masalah tersebut akan coba diatasi dalam tahap pemrosesan data, untuk saat ini yang dilakukan hanya akan menggabungkan dataset train dan val kemudian melakukan pemisahan (split) lagi.

In [ ]:
train = np.append(train, val, axis=0)
train, val = train_test_split(train, test_size=.20, random_state=777)

Untuk mengakhiri bagian ini, akan ditampilkan beberapa contoh dalam dataset yang dimiliki.

In [ ]:
plt.figure(figsize=(10, 10))
for k, i in np.ndenumerate(np.random.randint(train.shape[0], size=9)):
    ax = plt.subplot(3, 3, k[0] + 1)
    plt.imshow(train[i][0], cmap='gray')
    plt.title(labels[train[i][1]])
    plt.axis("off")

# Processing Data

pada tahap awal ini akan membuat dan menggunakan fungsi yang disebut prepared_data () yang akan menormalkan gambar (membagi setiap piksel dengan 255) dan me-reshape array menjadi sesuai bentuk. Setelah itu, fungsi akan mengembalikan array x dan y secara terpisah dari set kita.

In [ ]:
def prepare_data(data):
    x = []
    y = []
    
    for feature, label in data:
        x.append(feature)
        y.append(label)
        
    x = (np.array(x) / 255).reshape(-1,IMG_WIDTH, IMG_HEIGHT, 1)
    y = np.array(y)
        
    return x, y

x_train, y_train = prepare_data(train)
x_val, y_val = prepare_data(val)
x_test, y_test = prepare_data(test)

Untuk mencari performa terbaik dari model yang dilakukan, penting untuk menambah jumlah sampel dalam dataset dan untuk itu perlu melakukan proses data augumentasi. Perhatikan bahwa di sini tidak akan menggunakan flip untuk menghasilkan gambar baru karena paru-paru tidak simetris secara horizontal dan vertikal sehingga tidak diperlukan.

kemudian juga ditampilkan hasil dari data augmentasi yang dilakukan

In [ ]:
from tensorflow.keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range = 20, 
    zoom_range = 0.2, 
    width_shift_range=0.15,  
    height_shift_range=0.15,
    horizontal_flip = False,  
    vertical_flip=False)


datagen.fit(x_train)

# pick an image to transform
image_path = '/content/chest_xray/val/NORMAL/NORMAL2-IM-1427-0001.jpeg'
img = image.load_img(image_path)


img=image.img_to_array(img)
img=img.reshape((1,) + img.shape)

i = 0

for batch in datagen.flow(img, save_prefix='test', save_format='jpeg'):  # this loops runs forever until we break, saving images to current directory with specified prefix
    plt.figure(i)
    plot = plt.imshow(image.img_to_array(batch[0]).astype(np.uint8))
    i += 1
    if i > 4: 
        break

plt.show()

Nah, untuk menyelesaikan masalah ketidakseimbangan data yang disebutkan sebelumnya. Ada beberapa kemungkinan pendekatan untuk diambil tetapi kami akan memilih untuk memberikan weight (bobot/pembebanan) yang berbeda ke kelas.

weight ini akan digunakan di pada proses lanjutan sebagai parameter yang sesuai dengan model, dan sebagaimana dijelaskan dalam dokumentasi resmi Keras, fungsi compute_class_weight() dapat berguna untuk memberi tahu model agar "lebih memperhatikan" sampel dari under-represented class.


In [ ]:
class_weights = compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(y_train),
                                        y = y_train                                                    
                                    )
class_weights = dict(zip(np.unique(y_train), class_weights))
class_weights

#Membuat Model

In [ ]:
def block(inputs, filters, stride):
    conv_0 = layers.Conv2D(filters=filters, kernel_size=(3, 3), strides=(stride, stride), padding='same', activation='relu')(inputs)
    conv_1 = layers.Conv2D(filters=filters, kernel_size=(3, 3), strides=(stride, stride), padding='same', activation='relu')(conv_0)
    
    skip = layers.Conv2D(input_shape=input_size, filters=filters, kernel_size=(1, 1), strides=(stride**2, stride**2), padding='same', activation='relu')(inputs)
    
    pool = layers.MaxPool2D(pool_size=(3, 3), strides=(2,2), padding='same')(add([conv_1, skip]))
    
    return pool